In [1]:
import numpy as np
# Define solution and its Laplace
a = 4

def u(x, a):
  return np.sin(np.pi * a * x)

def u_xx(x, a):
  return -(np.pi * a)**2 * np.sin(np.pi * a * x)

# Define computional domain
bc1_coords = np.array([[0.0],
                       [0.0]])

bc2_coords = np.array([[1.0],
                       [1.0]])

dom_coords = np.array([[0.0],
                       [1.0]])

# Training data on u(x) -- Dirichlet boundary conditions

nn  = 100

X_bc1 = dom_coords[0, 0] * np.ones((nn // 2, 1))
X_bc2 = dom_coords[1, 0] * np.ones((nn // 2, 1))
X_u = np.vstack([X_bc1, X_bc2]) # data for BC
Y_u = u(X_u, a) # y data for BC

X_r = np.linspace(dom_coords[0, 0],
                  dom_coords[1, 0], nn)[:, None] # data for residual
Y_r = u_xx(X_r, a) # y data for residual

In [2]:
import train_NN as train
import neural_network as net
import torch
import utilities

# Define model
seed = 1
model = net.PINN(2,5) 
# model_params = []
# for param in model.parameters():
#     model_params.append(param.view(-1))
# model_params = torch.cat(model_params) 
mean = X_r.mean(axis=0)  # (1, dim) values
std = X_r.std(axis=0)  # (1, dim) values   
X_u_normal = (X_u - mean) / std
X_r_normal = (X_r - mean) / std
# Outputs are not normalized
# Convert data to Pytorch tensors requiring gradient
X_u_normal_tor = torch.tensor(X_u_normal, requires_grad=True)
X_r_normal_tor = torch.tensor(X_r_normal, requires_grad=True)
Y_u_tor = torch.tensor(Y_u)
Y_r_tor = torch.tensor(Y_r)

## tesing J_r calculation

In [3]:
Y_r_pred = model(X_r_normal_tor)
residual = train.calculate_residual(Y_r_pred, X_r_normal_tor)

n_params = sum([np.prod(p.size()) for p in model.parameters()])
n_r_batchsize = residual.shape[0]
J_r = torch.zeros(size=(n_params, n_r_batchsize))

for ind in range(n_r_batchsize):
    model.zero_grad()
    residual[ind, 0].backward(retain_graph=True)
    g_flat = []
    for param in model.parameters():
        if param.grad is not None:
            g_flat.append(param.grad.view(-1))
        else:
            g_flat.append(torch.zeros(size=param.size()).view(-1))
    g_flat = torch.cat(g_flat)
    J_r[:, ind] = g_flat 

In [4]:
J_r

tensor([[ 0.0041,  0.0041,  0.0040,  ...,  0.0050,  0.0056,  0.0061],
        [ 0.0023,  0.0029,  0.0037,  ...,  0.0286,  0.0292,  0.0300],
        [-0.1934, -0.1835, -0.1702,  ...,  0.0153,  0.0151,  0.0149],
        ...,
        [ 0.0467,  0.0495,  0.0523,  ..., -0.0489, -0.0419, -0.0348],
        [ 0.0088,  0.0094,  0.0099,  ...,  0.1196,  0.1229,  0.1252],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [5]:
def calculate_eigenvalues_from_j(j1, j2):
    j1_np = j1.detach().numpy()
    j2_np = j2.detach().numpy()
    K_matrix = j1.T@j2
    eigs = np.linalg.eigvalsh(K_matrix)
    # Return eigenvalues in descenind order
    return eigs[::-1]

calculate_eigenvalues_from_j(J_r, J_r)


array([ 1.50177318e+02,  8.01607480e+01,  5.63256210e+01,  4.09411066e+01,
        7.30269231e+00,  2.39001680e+00,  9.29207716e-01,  3.10755430e-01,
        2.29198125e-01,  2.50950381e-02,  2.99007740e-03,  1.10298091e-03,
        5.65815912e-05,  3.36923794e-05,  1.60640560e-05,  3.90618012e-06,
        6.25436186e-07,  1.15774270e-07,  4.48827164e-08,  9.68194296e-09,
        1.99313340e-09,  2.53837555e-10,  5.35434303e-11,  1.81326600e-11,
        5.56632254e-12,  1.33514259e-12,  1.36480500e-13,  4.31056334e-14,
        2.42268505e-14,  1.44726351e-14,  1.14197671e-14,  9.60629445e-15,
        8.98412190e-15,  8.39286030e-15,  7.09160670e-15,  6.76042494e-15,
        6.29075478e-15,  6.08391280e-15,  5.21054352e-15,  5.06056735e-15,
        4.38223303e-15,  4.17577695e-15,  3.74818610e-15,  3.57543230e-15,
        3.28982902e-15,  3.10772230e-15,  3.01465765e-15,  2.55625327e-15,
        2.37609102e-15,  2.15017422e-15,  2.02618178e-15,  1.74793487e-15,
        1.48793273e-15,  

In [12]:
Y_r_pred = model(X_r_normal_tor)
residual = train.calculate_residual(Y_r_pred, X_r_normal_tor)
J_r = utilities.calculate_j_r(model, residual)
J_r

tensor([[ 0.0041,  0.0041,  0.0040,  ...,  0.0050,  0.0056,  0.0061],
        [ 0.0023,  0.0029,  0.0037,  ...,  0.0286,  0.0292,  0.0300],
        [-0.1934, -0.1835, -0.1702,  ...,  0.0153,  0.0151,  0.0149],
        ...,
        [ 0.0467,  0.0495,  0.0523,  ..., -0.0489, -0.0419, -0.0348],
        [ 0.0088,  0.0094,  0.0099,  ...,  0.1196,  0.1229,  0.1252],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

## Tesing J_u calculation - works well

In [10]:
Y_u_pred = model(X_u_normal_tor)  # (no_of_points, 1)
J_u = utilities.calculate_j_u(model, Y_u_pred)
J_u

tensor([[ 0.0033,  0.0033,  0.0033,  ...,  0.0302,  0.0302,  0.0302],
        [-0.0361, -0.0361, -0.0361,  ...,  0.0356,  0.0356,  0.0356],
        [-0.1537, -0.1537, -0.1537,  ...,  0.0162,  0.0162,  0.0162],
        ...,
        [-0.7824, -0.7824, -0.7824,  ..., -0.6711, -0.6711, -0.6711],
        [-0.8880, -0.8880, -0.8880,  ..., -0.7094, -0.7094, -0.7094],
        [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000]])

In [20]:
n_params = sum([np.prod(p.size()) for p in model.parameters()])
J_u_no_retain = torch.zeros(size=(n_params, X_r_normal_tor.shape[0]))
for ind in range(X_u_normal_tor.shape[0]):
    Y_r_pred = model(X_r_normal_tor)  # (no_of_points, 1)
    g = torch.autograd.grad(
              Y_r_pred[ind,0], model.parameters(), 
              grad_outputs=torch.ones_like(Y_r_pred[ind,0]),
              retain_graph=False,
              create_graph=False
              )
    g_new = []
    for i in g:
        g_new.append(i.view(-1))
    g_new = torch.cat(g_new)
    J_u_no_retain[:,ind] = g_new

In [22]:
Y_r_pred = model(X_r_normal_tor)  # (no_of_points, 1)
n_params = sum([np.prod(p.size()) for p in model.parameters()])
J_u_retain = torch.zeros(size=(n_params, X_r_normal_tor.shape[0]))
for ind in range(X_u_normal_tor.shape[0]):
    g = torch.autograd.grad(
              Y_r_pred[ind,0], model.parameters(), 
              grad_outputs=torch.ones_like(Y_r_pred[ind,0]),
              retain_graph=True,
              create_graph=False
              )
    g_new = []
    for i in g:
        g_new.append(i.view(-1))
    g_new = torch.cat(g_new)
    J_u_retain[:,ind] = g_new

In [23]:
J_u_no_retain[:10,5]

tensor([ 4.0098e-03, -3.5740e-02, -1.9290e-01,  1.8028e-04, -2.4537e-03,
        -2.6011e-03,  2.3184e-02,  1.2513e-01, -1.1694e-04,  1.5917e-03])

In [24]:
J_u_no_retain[:10,10]

tensor([ 4.8203e-03, -3.5172e-02, -2.3487e-01,  2.9750e-04, -3.0399e-03,
        -3.5226e-03,  2.5703e-02,  1.7164e-01, -2.1741e-04,  2.2215e-03])

In [25]:
J_u_retain[:10,10]

tensor([ 4.8203e-03, -3.5172e-02, -2.3487e-01,  2.9750e-04, -3.0399e-03,
        -3.5226e-03,  2.5703e-02,  1.7164e-01, -2.1741e-04,  2.2215e-03])

In [2]:
import train_NN as train
import neural_network as net
# Define model
seed = 1
model = net.PINN(2,5) 
# X = np.random.normal(size=(15,1))
# Y = model(torch.from_numpy(X))
# for p in model.named_parameters():
#     print(p[0])
dets = train.train_nn_model(model, train_data=(X_u, Y_u, X_r, Y_r),
                            no_iterations=1000)
dets['Loss']

[12298.587851020546,
 12298.313750463853,
 12298.03853590243,
 12297.762193822451,
 12297.484710647763,
 12297.206072738692,
 12296.926266390905,
 12296.645277834185,
 12296.363093231279,
 12296.0796986767,
 12295.795080195527,
 12295.509223742209,
 12295.222115199373,
 12294.9337403766,
 12294.644085009224,
 12294.35313475711,
 12294.060875203433,
 12293.76729185345,
 12293.472370133271,
 12293.176095388615,
 12292.878452883579,
 12292.579427799377,
 12292.279005233097,
 12291.97717019644,
 12291.67390761446,
 12291.369202324278,
 12291.063039073832,
 12290.755402520583,
 12290.446277230229,
 12290.13564767541,
 12289.823498234431,
 12289.50981318992,
 12289.194576727563,
 12288.877772934758,
 12288.559385799297,
 12288.239399208047,
 12287.917796945621,
 12287.594562693017,
 12287.269680026275,
 12286.943132415154,
 12286.614903221727,
 12286.284975699047,
 12285.953332989757,
 12285.619958124718,
 12285.284834021615,
 12284.947943483568,
 12284.609269197728,
 12284.268793733863,
 12